# Matrix template class

Here I will provide some brief overview about the `Matrix` template class. All the subjects are heavily based on the [official documentation](https://eigen.tuxfamily.org/dox/group__TutorialMatrixClass.html).

The following dependencies will be used:

In [1]:
#include <iostream>
#include <eigen3/Eigen/Dense>

using namespace Eigen;

## The first three template parameters of `Matrix`

`Matrix` class takes 6 templates parameters. But for now only the first three will be necessary. This parameters are:

```C++
Matrix<typename Scalar, int RowsAtCompileTime, int ColsAtCompileTime>
```

Furthermore:

* `Scalar` is the type of the entries (coefficients);

* `RowsAtCompileTime` and `ColsAtCompileTime` are the number of rows and columns of the matrix defined at compile tome.

For very frequent cases, Eigen provides convenience `typedef`s. For 4x4 matrix and 3x3 matrix, for example, one can simply do:

```C++
Matrix4d matrix_1;
Matrix3d matrix_2;
```

Behind the scenes, this is what is happening:

```C++
typedef Matrix<double, 4, 4> Matrix4d;
typedef Matrix<double, 3, 3> Matrix3d;
```

## Vectors

Vectors are just a particular case of `Matrix`. They have either 1 row or 1 column. In Eigen, the most common is vectors as 1-column, or, better saying, column-vectors. Otherwise, they are called row-vectors.

Analogously for `Matrix`s, Eigen provide convenience methods for vectors. For example:

```C++
typedef Matrix<float, 3, 1> Vector3f;  // column-vector with 3 float entries
```

There are also row-vectors, say:

```C++
typedef Matrix<int, 1, 2> RowVector2i;  // row-vector with 2 integer entries
```

## Dynamic sizes

All the previous case are called __fixed-size__, since one states the size of the `Matrix` at compile time. It is also possible to have __dynamic size__, which are determined at run-time. To do so, Eigen uses a special value called `Dynamic`. The combination of this value with `Matrix` produces the following convenience typedef, for example:

```C++
typedef Matrix<double, Dynamic, Dynamic> MatrixXd;
typedef Matrix<float, Dynamic, Dynamic> MatrixXf;
typedef Matrix<int, Dynamic, Dynamic> MatrixXi;
```

or, analogously for vectors, we have

```C++
typedef Matrix<double, Dynamic, 1> VectorXd;
typedef Matrix<float, Dynamic, 1> VectorXf;
typedef Matrix<int, Dynamic, 1> VectorXi;
```

Moreover, one can also have a fixed number of rows and dynamic column size just as

```C++
Matrix<double, 4, Dynamic>
```

There are other combinations, but above I provided the most useful for me.

## Constructors

Constructors are always available and can be easily used just as:

```C++
Matrix3f a;
MatrixXf b;
```

where

* `a` is a unitialized 3x3 matrix for float entries;
* `b` is a unitialized dynamic-size matrix. It is currently a 0x0 matrix and its coefficients hasn't yet been allocated.

Alternatively, we could initialize with allocate array of coefficientes with a given size, for instance

```C++
MatrixXf c(10, 15);
VectorXf d(15);
```

where  the first matrix index is the number of rows and the last are columns. Furthermore

* `c` is a 10x15 dynamic-size matrix with allocated but currently unitialized coefficients;
* `d` is a dynamic-size vector of size 30 with allocated but currently unitialized coefficients

Eigen offers constructors to initialize small fixed-size vectors up to size 4:

```C++
Vector2d a(1.0, 2.0);
Vector3d b(1.0, 2.0, 3.0);
Vector4d c(1.0, 2.0, 3.0, 4.0);
```

## Element accessing

To access elements in Matrix and Vector Eigen objects is pretty simple. As general rule, always use `operator()`, this will work as expected for both matrix and vector. Let's see it in practice:

In [2]:
Matrix2d matrix_1 = Matrix2d::Random();

std::cout << matrix_1;

-0.207313  0.483821
 0.248325  -0.60754

We can simply access $(\text{row}, \text{col})$, say, for `row = 1` and `col = 0`, as

In [3]:
matrix_1(1, 0)

0.248325

For vectors, the same is valid:

In [4]:
Vector3f vec_1 = Vector3f::Random();

std::cout << vec_1;

-0.737212
 0.473095
-0.657211

In [5]:
vec_1(1)

0.4731f

Or even with

In [6]:
vec_1[1]

0.4731f

As a final remark on this topic, for matrix the accessing way `matrix(index)` works, but caution must be taken. This result in a index-based access in the array of coefficients. So, if you try `matrix(2)`, you will access the third element stored in `matrix`. By default, Eigen uses column-major storage order. It means that index-based numeration is performed over the columns, before from top to bottom, after left to right. Taking `matrix_1` as example:

In [7]:
matrix_1(2)

0.483821

And the enumeration is:

| index | matrix entry     |
|:---   |     :---:        |
| 0     |    (0, 0)        |
| 1     |    (1, 0)        |
| 2     |    (0, 1)        |
| 3     |    (1, 1)        |

## Initialization

Matrix and vector coefficients can be initialized with a convenience comma-initialization fashion (see below).

In [8]:
Matrix3d matrix_2;
matrix_2 << 1, 2, 3,
            4, 5, 6,
            7, 8, 9;

std::cout << matrix_2;

1 2 3
4 5 6
7 8 9

## Size and resize

The size of a matrix can be retrieved by the method `size()`:

In [9]:
std::cout << matrix_2.size();

9

which returns the number of entries (coefficients). It is equivalent to the product `rows * columns`. One can obtain the number of rows and columns as follows:

In [10]:
std::cout << "Number of rows: " << matrix_2.rows() << std::endl;
std::cout << "Number of columns: " << matrix_2.cols();

Number of rows: 3
Number of columns: 3

If you want to resize a dynamic-size `Matrix`, you just can use `resize()` method:

In [11]:
Matrix<double, Dynamic, Dynamic> matrix_3(2, 4);
matrix_3 << 1, 2, 3, 4,
            5, 6, 7, 8;

In [12]:
std::cout << matrix_3;

1 2 3 4
5 6 7 8

In [13]:
matrix_3.resize(4, 2);

std::cout << matrix_3;

1 3
5 7
2 4
6 8

There are two remarks:

1. `resize()` does not work on fixed-size `Matrix`, unless the desired size is the same as the original. In practical terms, it means that there is no effect in resizing a fixed-size `Matrix`, but it is still available for the sake of API uniformity.
2. For dynamic-size, the `Matrix` storage is column-major. So, the `resize()` follows the storage order.

If you want a row-major storage, you can use the template arguments as:

In [14]:
Matrix<double, Dynamic, Dynamic, RowMajor> matrix_4(2, 4);
matrix_4 << 1, 2, 3, 4,
            5, 6, 7, 8;

std::cout << matrix_4;

1 2 3 4
5 6 7 8

In [15]:
matrix_4.resize(4, 2);

std::cout << matrix_4;

1 2
3 4
5 6
7 8

## Assignment

For definition, assignment consists in copying a matrix into another by means of `operator=`. If you assign to a left-hand (dynamic-size) matrix another (dynamic-size) one with different size, Eigen automatically converts the size of the left-hand matrix to match the size of the right-hand matrix. See below:

In [16]:
MatrixXd matrix_5(2, 2);

std::cout << "matrix_5 size: " << matrix_5.rows() << "x" << matrix_5.cols();

matrix_5 size: 2x2

In [17]:
MatrixXd matrix_6(3, 3);
matrix_5 = matrix_6;

std::cout << "matrix_5 size: " << matrix_5.rows() << "x" << matrix_5.cols();

matrix_5 size: 3x3

Note that if the left-hand matrix is of fixed-size, resize will be not allowed.

## Fixed vs. Dynamic size

As commented in the Eigen's documentation, it is recommended to use fixed size matrix (like `Matrix4f`) when you can and for small sizes. Otherwise, use dynamic size. For small sizes (say, less than 16, for example), fixed-size `Matrix` performs better than dynamic-size. However, around size 32 and above, the differences between performances are negigible.

Another important point is that creating large fixed-size `Matrix`, say, inside a function, can result in a stack overflow, since the variable will be local, and it is commonly stored on the stack.

## `Matrix` optional template parameters

Now, let's learn a little about the remaining 3 template parameters of `Matrix`. In the beginning of this notebook, we saw the first three parameters. The last three parameters are optional, but they can be useful in some cases. See below the complete list of template parameters:

```C++
Matrix<typename Scalar,
       int RowsAtCompileTime,
       int ColsAtCompileTime,
       int Options = 0,
       int MaxRowsAtCompileTime = RowsAtCompileTime,
       int MaxColsAtCompileTime = ColsAtCompileTime>
```

About the three last parameters:

* `Options` specifies the type of storage order. There we can select a `RowMajor` storage or a `ColMajor`.
* `MaxRowsAtCompileTime` and `MaxColsAtCompileTime` define fixed upper bounds for the number of rows and columns, even when the exact size of the `Matrix` is not known at compile time.

## Convenience `typedef`s

We already saw some `typedef`s around. Let's see `typedef`s that Eigen provides in a bigger picture:

* `MatrixNt` for `Matrix<type, N, N>`. For example, `MatrixXi` for `Matrix<int, Dynamic, Dynamic>`.
* `VectorNt` for `Matrix<type, N, 1>`. For example, `Vector2f` for `Matrix<float, 2, 1>`.
* `RowVectorNt` for `Matrix<type, 1, N>`. For example, `RowVector3d` for `Matrix<double, 1, 3>`.

Where:

* `N` can be any one among 2, 3, 4 or X (Dynamic-size).
* `t` can be any one of `i` (int), `f` (float), `d` (double), `cf` (`complex<float>`) or `cd` (`complex<double>`). There are more Scalar options supported, but the aforementioned are the ones we need here.